project-1 
population Statistic
population DataFrames
population Pyramids
Animated Population Pyramids
Population Maps

Data
Using wbdata

for data: year, people/males/females, world/region/country

In [1]:

import wbdata
import numpy as np
import pandas as pd 


def population(year = '2017',people='FE',age_range = '7579',country = 'CHN'):
    """ this function takes country,year age_range,people:'FE','MA','TO'
    returns the population of peopel who live in the country in the year with age 
    between the age_range
    """
    #country = wbdata.search_countries(country)
    variable = 'SP.POP.'+age_range+'.'+people
    variable_labels = {variable: "Population"}
    dat = wbdata.get_dataframe(variable_labels, country)
    #get the dataframe from world bank.
    dat.index = dat.index.astype(int)
    year = int(year)
    #convert the type into int
    count = dat.at[year,'Population']
    return count

In [2]:
population(year='2017', people = 'FE',age_range = '7579', country = 'CHN')
## this function will return the female population in China 2017 with the age range 75-70

12870257.0

In [3]:
population(year='2016', people = 'MA',age_range = '1519', country = 'WLD')
# it will return the male population with age between 15-19 in world in 2016 

306249453.0

In [4]:

import pandas as pd
def pop_data(year = ['2016','2017'], country = ['WLD','CHN']):
    """this function take year which may be a list and one country
    the output is a dataframe, which is indexed by country and year
    with columns providing the different age-sex group"""
    
    
    age_range = []
    for i in range(0,80,5):
        age_range.append(f"{i:02d}"+f"{i+4:02d}")
    age_range.append("80UP")
    # create the age_range list

    col_index = [['Female']*len(age_range)+['Male']*len(age_range),age_range*2]
    index = [[i for i in country for r in range(len(year))],year*len(country)]
    dat = pd.DataFrame(columns = col_index,
                       index = index)
    #create the dataframe
    
    def get_year_pop(year,country):
        pop_FE = [population(year = year, people = 'FE',age_range = i, country=country) for i in age_range]
        pop_MA = [population(year = year, people = 'MA',age_range = i, country=country) for i in age_range]
        # list comprehension
        pop_dat = pop_FE+pop_MA
        return pop_dat
    # get population number from population function with specific country and year 
    
    for country_i in country:
        for year_i in year:
            dat.loc[country_i, year_i] = get_year_pop(year_i,country_i)
    # replace the value of dataframe by row
    return dat




In [5]:
pop_data(year = ['2015','2016','2017'],country = ['CHN','WLD'])
# it will return a sex-age population dataframe from World and Chian in both 2016 and 2017

Female                                                         \
               0004       0509       1014       1519       2024       2529   
CHN 2015   39060174   37305644   35893145   37164848   47056442   61510279   
    2016   39148835   37730861   36090468   36277938   44198772   60611149   
    2017   38905936   38191085   36362097   35847781   41843853   58007050   
WLD 2015  324590277  310458468  294913382  286382889  290755124  298403783   
    2016  325948095  313401086  297513705  286801705  288725344  299141212   
    2017  326937386  316063452  300382891  287858778  287109865  297839524   

                                                        ...          Male  \
               3034       3539       4044       4549    ...          3539   
CHN 2015   48738459   45932720   56866505   60139220    ...      48370065   
    2016   51359118   45123940   54720949   60627169    ...      47519911   
    2017   54823636   44989116   52198476   60492029    ...      47378056   
WLD 2015  271343729  246208159  239005649  225534858    ...     252655574   
    2016  277329697  249389789  239886740  228535031    ...     255721917   
    2017  283727043  253526812  240343273  231150172    ...     259847867   

                                                                            \
               4044       4549       5054       5559       6064       6569   
CHN 2015   59640967   61934721   51119819   39918337   39363451   25584174   
    2016   57501799   62510811   53515012   40905774   39658939   27584196   
    2017   54896967   62566820   55958535   42691551   39125496   30116848   
WLD 2015  244804578  227917661  200611430  167439894  142739712  102869510   
    2016  245598208  231062337  205279529  171361554  146002874  107749126   
    2017  245830428  233993742  209768271  176179378  148381780  113285604   

                                        
              7074      7579      80UP  
CHN 2015  16955317  11720249   9764927  
    2016  17465978  11855996  10429878  
    2017  18092070  11948600  11011116  
WLD 2015  71297196  50814071  48288575  
    2016  73451684  51528907  51079885  
    2017  76136197  52021968  53547881  

[6 rows x 34 columns]

In [6]:
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np

def pop_pyramid(data, country = 'CHN', year = ['2015','2016']):
    """this is a function taking pandas dataframe
    output is population pyramid graph with specific country"""
    py.init_notebook_mode(connected=True)
    
    age_range = []
    for i in range(0,80,5):
        age_range.append(f"{i:02d}"+f"{i+4:02d}")
    age_range.append("80UP")
    # create the age_range list 
    
    layout = go.Layout(barmode='overlay',
                   yaxis=go.layout.YAxis(range=[0, 90], title='Age'),
                   xaxis=go.layout.XAxis(title='Number'))
    # set the plot
    
    bins = [go.Bar(x = np.array(data.loc[country,year_i]['Female']),
               y = [int(s[:2])+1 for s in age_range],
               orientation='h',
               name='Men {0:d}'.format(int(year_i)),
               hoverinfo='skip'
              )
            for year_i in year]
          
    bins += [go.Bar(x = -np.array(data.loc[country, year_i]['Male']),
                    y=[int(s[:2])+1 for s in age_range],
                    orientation='h',
                    name='Women {0:d}'.format(int(year_i)),
                    hoverinfo='skip',
                   )
             for year_i in year]
    # plot

    return py.iplot(dict(data=bins, layout=layout))    

In [7]:
#get the population data from China, USA and World in year 1997,2007,2017
years = ['1997','2007','2017']
dat = pop_data(year = years, country = ['CHN','USA','WLD'])
dat

Female                                                         \
               0004       0509       1014       1519       2024       2529   
CHN 1997   38382219   60823448   55451735   45952516   53345437   62028701   
    2007   37038310   35874064   42384005   58737862   55562056   45641820   
    2017   38905936   38191085   36362097   35847781   41843853   58007050   
USA 1997    9802914    9898595    9605913    9209420    9097679    9516810   
    2007   10292563    9756595   10370161   10662607   10336408    9898295   
    2017    9676632   10158892   10279316   10423372   11177860   11358292   
WLD 1997  292544989  307495210  291269881  262356914  249956558  242001774   
    2007  306380447  290739688  290177006  301629307  287767925  257874916   
    2017  326937386  316063452  300382891  287858778  287109865  297839524   

                                                        ...          Male  \
               3034       3539       4044       4549    ...          3539   
CHN 1997   56114719   43394029   42001893   35265873    ...      45754283   
    2007   52986668   61587698   55582878   42791677    ...      64017063   
    2017   54823636   44989116   52198476   60492029    ...      47378056   
USA 1997   10602592   11274328   10679106    9512752    ...      11489965   
    2007    9796110   10099857   10899842   11340514    ...      10281254   
    2017   10880538   10314787   10074535   10196272    ...      10356373   
WLD 1997  222461431  194944256  175345825  148241821    ...     200395634   
    2007  245224029  237089081  217183958  188734703    ...     243157046   
    2017  283727043  253526812  240343273  231150172    ...     259847867   

                                                                            \
               4044       4549       5054       5559       6064       6569   
CHN 1997   44108667   36842569   26201017   22954917   20518690   15986100   
    2007   57805681   44860108   42762528   35025468   23876251   19384888   
    2017   54896967   62566820   55958535   42691551   39125496   30116848   
USA 1997   10628383    9243437    7280103    5759100    4846002    4391978   
    2007   11135783   11326275   10363028    8925588    6813674    5117255   
    2017    9996045   10259490   10868379   10808942    9601635    7939154   
WLD 1997  179312062  149498921  115926247  100544486   86351857   68440810   
    2007  221435111  190857272  167566564  136361105  101259893   81183290   
    2017  245830428  233993742  209768271  176179378  148381780  113285604   

                                        
              7074      7579      80UP  
CHN 1997  10735377   6676030   4613154  
    2007  15132986   9388576   7028466  
    2017  18092070  11948600  11011116  
USA 1997   3884740   2884206   2899984  
    2007   3976954   3129654   3851707  
    2017   5728630   3868301   4825690  
WLD 1997  48688883  29007574  24444398  
    2007  62283183  41321404  35727969  
    2017  76136197  52021968  53547881  

[9 rows x 34 columns]

In [8]:
#draw the animated population pyramids with one country and selected years
pop_pyramid(data=dat, country = 'USA', year = ['2017','2007','1997'])